In [1]:
from model_trainer import ModelTrainer
from test_performance import TestPerformance, DatasetPath, MMLU_Subset

/home/sheju347/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os

trainer = ModelTrainer()

# model_name = "microsoft/Phi-3-mini-4k-instruct"
# model_name = os.path.abspath("../../../projects/sciences/computing/sheju347/MedicalQA/train/saved_models/fine_tuned_model_entire_UltraMedical_batch_4")
model_name = "/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/fine_tuned_model_entire_UltraMedical_batch_4"
# model_name = "google/flan-t5-xl"

trainer.load_model(model_name)
# trainer.load_model_t5(model_name)
# trainer.test_model_MedQA_response()



---------- start checking GPU -----------
GPU: NVIDIA H100 NVL
torch.cuda.is_bf16_supported():  True
---------- finish checking GPU -----------
---------- start loading model:/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/fine_tuned_model_entire_UltraMedical_batch_4 -----------
finish loading tokenizer


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.11it/s]


finish loading model
torch_dtype: torch.float32
cuda available: True
device: cuda
---------- finish loading model:/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/fine_tuned_model_entire_UltraMedical_batch_4 -----------


In [ ]:
https://ondemand.otago.ac.nz/home/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/fine_tuned_model_entire_UltraMedical_batch_4

In [ ]:

test = TestPerformance(trainer.model, trainer.tokenizer, trainer.device, is_encoder_decoder = False)
# test.test_accuracy(DatasetPath.MMLU, subset_name = MMLU_Subset.College_medicine, is_ensemble = False)
# data_range = [9, 13, 19, 24, 28, 29, 30, 31, 32, 38, 43, 44, 45, 46, 52, 56, 57, 62, 64, 68, 71, 73, 77, 87, 90, 94, 96, 97, 108, 
#               109, 110, 118, 123, 126, 133, 135, 139, 141, 143, 144, 145, 146, 159, 160, 166, 167, 170, 171, 172, 173, 179, 180, 181, 183, 
#               185, 186, 188, 191, 196, 197, 203, 205]

#range(205, 206) # None #range(34, 35)

data_range = None

topK_searchEngine = 30
topK_monoBERT = 3

test.test_accuracy(DatasetPath.MedQA, subset_name = None, is_ensemble = False, use_RAG = True, data_range = data_range, 
                  topK_searchEngine = topK_searchEngine, topK_monoBERT = topK_monoBERT)
# test.test_MedQA_response(use_RAG = True)
# print(test.get_RAG_context("A junior orthopaedic surgery resident is completing a carpal tunnel repair with the department chairman as the attending physician. During the case, the resident inadvertently cuts a flexor tendon. The tendon is repaired without complication. The attending tells the resident that the patient will do fine, and there is no need to report this minor complication that will not harm the patient, as he does not want to make the patient worry unnecessarily. He tells the resident to leave this complication out of the operative report. Which of the following is the correct next action for the resident to take?"))
# test.

model: /projects/sciences/computing/sheju347/MedicalQA/train/saved_models/fine_tuned_model_entire_UltraMedical_batch_4


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


reranked top k score: [-0.8158773, -0.8480601, -1.3686457]
question 1/1273 answer:B correct_answer:B True
reranked top k score: [-4.3700304, -5.3764153, -5.825001]
question 2/1273 answer:D correct_answer:D True
reranked top k score: [-3.7423773, -4.066557, -5.0958023]
question 3/1273 answer:A correct_answer:B False
reranked top k score: [-1.6357648, -1.7770984, -2.155676]
question 4/1273 answer:D correct_answer:D True
reranked top k score: [-0.6621198, -0.7818354, -0.8809782]
question 5/1273 answer:B correct_answer:B True
reranked top k score: [2.6340055, 2.2501125, 1.5073938]
question 6/1273 answer:D correct_answer:D True
reranked top k score: [3.076138, 2.3729076, 2.1508884]
question 7/1273 answer:C correct_answer:C True
reranked top k score: [-0.97630084, -1.207248, -1.2318461]
question 8/1273 answer:C correct_answer:C True
reranked top k score: [-0.35251087, -0.6186423, -0.6682904]
question 9/1273 answer:D correct_answer:B False
reranked top k score: [2.1236079, 1.4422977, 0.900784

In [ ]:
prompt_RAG = '''
You are a medical question answering assistant.

The following context may or may not be useful. Use it only if it helps answer the question.
INSTRUCTIONS:
- If the context directly helps answer the question, use it and cite appropriately
- If the context is topically related but not diagnostically relevant, acknowledge it but rely on your medical knowledge
- If the context might mislead you toward a less likely diagnosis, explicitly state why you're not following it

Context:
{context}

Question:
{question}

{choices}
'''

def format_choices(choices):
    a = zip(list(choices.keys()), choices.values())
    final_answers = []
    for x,y in a:
        final_answers.append(f'[{x}] : {y}')
    return "\n".join(final_answers)
    

data = {"question": "A 34-year-old woman is assaulted and suffers a number of stab wounds to her abdomen. Bystanders call paramedics and she is subsequently taken to the nearest hospital. On arrival to the emergency department, her vitals are T: 36 deg C, HR: 110 bpm, BP: 100/60, RR: 12, SaO2: 99%. A FAST and abdominal CT are promptly obtained which are demonstrated in Figures A and B, respectively. Her chart demonstrates no other medical problems and vaccinations/boosters up to date. The patient is diagnosed with a Grade V splenic laceration and is immediately brought to the OR for emergent splenectomy. The splenectomy is successfully performed with removal of the damaged spleen (Figure C). Following the operation, the patient should receive which of the following vaccines: (I) H. influenzae (II) Tetanus (III) N. meningitidis (IV) S. pneumoniae (V) Hepatitis B",
        "choices": {
        "A": "I, II",
        "B": "I, III, IV",
        "C": "I, V",
        "D": "III, IV"
        },
        "correct": "B",
        "source": "Test data index 0 (GBaker/MedQA-USMLE-4-options)"
}


prompt = prompt_RAG
rag_1 = "Traumatic splenic laceration with delayed rupture secondary to coughing in a patient with Von Willebrand disease. We describe the case of a 54-year-old male with Von Willebrand Disease who presented to the Emergency Department (ED) with 2Â weeks of worsening abdominal pain after falling on his left flank while boating. On his initial presentation, he was found to have a Grade II splenic injury that was managed non operatively by the trauma service. Four days later, he returned to the ED when he developed severe abdominal pain after coughing and was found to have active extravasation from the splenic parenchyma with hemoperitoneum on CT angiography and a grossly positive FAST exam. Intraoperatively, he was found to have a Grade V splenic injury and subsequently underwent splenectomy."
rag_2 = "Development of Bilateral Heterotopic Ossification After Survival of Life Threatening Purpura Fulminans. Heterotopic ossification has been reported in patients who have undergone traumatic amputations, burn injuries, and total hip arthroplasty; however, the incidence of heterotopic ossification following purpura fulminans has only been reported in one case with unilateral involvement. Here we present a bilateral lower extremity case of heterotopic ossification as sequelae of purpura fulminans.Â  A 34-year-old male smoker with a past medical history of stab wounds to the chest and abdomen requiring emergent exploratory laparotomy, diaphragmatic repair, and splenectomy 15 years agoÂ presented to the emergency department with a rapid onset of high fevers, chills and myalgia. He did not receive post-splenectomy prophylactic vaccinations forÂ "
rag_3 = "Ectopic splenic autotransplantation following traumatic injury: A case report. A 41-year-old male patient was admitted to the General Hospital of Guangzhou Military Command due to upper abdominal pain persisting for 12 h. Computed tomography (CT) and positron emission tomography/CT scans revealed multiple soft-tissue shadows in the abdominal cavity, peritoneum and Glisson's capsule, but the metabolic activity was at normal levels. A small area of low-density shadows near the tail of the pancreas and multiple shadows of enlarged lymph nodes were identified around the porta hepatis and the pancreas, with a mildly increased metabolic activity. On the basis of the CT images the patient was diagnosed with pancreatitis. Radionuclide imaging showed the absence of the spleen from its normal position (following splenectomy), but abnormal phagocytosis of multiple red blood cells was observed in the abdomen, which was diagnosed as ectopic splenic autotransplantation (ESAT). The patient subsequently recovered well following symptomatic treatment. ESAT in trauma patients requires urgent surgery in order to remove the damaged spleen and artificially cultivate partial splenic tissue."

context = rag_1 + '\n\n' + rag_2 + '\n\n' + rag_3
#self.get_RAG_context(example["question"])
question = data["question"]
choices = data["choices"]
answer_key = data["correct"]
formated_choices = format_choices(choices)
model_prompt = prompt.format(context = context, question = question, choices = formated_choices)


messages = [{"role": "user", "content": f"{model_prompt}"}]
# add_generation_prompt indicates the start of a response
inputs = trainer.tokenizer.apply_chat_template(messages, add_generation_prompt = True, return_tensors = "pt").to(trainer.device)
# print("inputs:", tokenizer.apply_chat_template(messages, add_generation_prompt = True, tokenize = False))
generate_kwargs = {
    "max_new_tokens": 1024,
    "do_sample": True,
    "temperature": 0.00001,
}
outputs = trainer.model.generate(inputs, **generate_kwargs)

text = trainer.tokenizer.batch_decode(outputs)[0]
# output_text = run_inference(model_prompt, self.model, self.tokenizer, max_new_tokens = 1024, temperature = self.temperature)
# output_text = output_text.split("<|assistant|>")[-1]
print("model output:", text)
